<a href="https://colab.research.google.com/github/tsly123/CNU_code/blob/master/DCGAN/Train_DCGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**READ ME FIRST**

This code is taken from https://github.com/tatsy/keras-generative with small modifications. Visit the github for more GAN models.

- This notebook is for initiating the training **DCGAN** with **CelebA dataset** (image size 32x32). It also contains **notes and troubleshooting instruction**. The main code is in folder models.
- Download ** folder model + Train_DCGAN.ipynb + celebA_32.hdf5** to your Drive and run with Colab.
- The image results will be saved in `./output/dcgan/results/`.
- The example results are in folder `./output32_example/dcgan/results/`.
- Another option is that you can download the folder `PC_code` and  `celebA_32.hdf5` or `celebA_64.hdf5` and run with your PC. Your PC should have GPU + >=12GB RAM or >=24GB RAM for running with `celebA_32.hdf5` or `celebA_64.hdf5`, respectively.

tsly, Sat Dec 8 16:34:49 2018

In [0]:
# Mount the Drive
import os

if os.path.exists('./MyDrive') == False:
  # Install a Drive FUSE wrapper.
  # https://github.com/astrada/google-drive-ocamlfuse
  !apt-get install -y -qq software-properties-common python-software-properties module-init-tools
  !add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
  !apt-get update -qq 2>&1 > /dev/null
  !apt-get -y install -qq google-drive-ocamlfuse fuse

  # Generate auth tokens for Colab
  from google.colab import auth
  auth.authenticate_user()

  # Generate creds for the Drive FUSE library.
  from oauth2client.client import GoogleCredentials
  creds = GoogleCredentials.get_application_default()
  import getpass
  !google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
  vcode = getpass.getpass()
  !echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

  # Create a directory and mount Google Drive using that directory.
  !mkdir -p MyDrive
  !google-drive-ocamlfuse MyDrive

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package libfuse2:amd64.
(Reading database ... 26397 files and directories currently installed.)
Preparing to unpack .../libfuse2_2.9.7-1ubuntu1_amd64.deb ...
Unpacking libfuse2:amd64 (2.9.7-1ubuntu1) ...
Selecting previously unselected package fuse.
Preparing to unpack .../fuse_2.9.7-1ubuntu1_amd64.deb ...
Unpacking fuse (2.9.7-1ubuntu1) ...
Selecting previously unselected package google-drive-ocamlfuse.
Preparing to unpack .../google-drive-ocamlfuse_0.7.1-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Setting up libfuse2:amd64 (2.9.7-1ubuntu1) ...
Processing triggers for libc-bin (2.27-3ubuntu1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Setting up fuse (2.9.7-1ubuntu1) ...
Setting up google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Please, open the following URL in a web browser: https://accounts.google

In [0]:
# check if the code and weight are in the same directory. Assume the code and weight folder name "DLtutorial"
# and MyDrive is the parent folder of your Google Drive.
# Otherwise, please check the directory again by !echo $pwd

!ls MyDrive/DLtutorial/DCGAN

 celebA_32.hdf5   models	     PC_code	  'READ ME FIRST.odt'
 celebA_64.hdf5   output32_example   __pycache__   Train_DCGAN.ipynb


In [0]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import numpy as np
import matplotlib
import h5py
matplotlib.use('Agg')

from MyDrive.DLtutorial.DCGAN.models import DCGAN

models = {'dcgan': DCGAN}

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/usr/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.6/dist-packages/traitlets/config/application.py", line 657, in launch_instance
    app.initialize(argv)
  File "<decorator-gen-121>", line 2, in initialize
  File "/usr/local/lib/python3.6/dist-packages/traitlets/config/application.py", line 87, in catch_config_error
   

In [0]:
def load_data(filename = 'MyDrive/DLtutorial/DCGAN/celebA_32.hdf5', size=-1):
    f = h5py.File(filename)

    dset = Dataset()
    dset.images = np.asarray(f['images'], 'float32') / 255.0

    if size > 0:
        dset.images = dset.images[:size]

    return dset
  
class Dataset(object):
    def __init__(self):
        self.images = None

    def __len__(self):
        return len(self.images)

    def _get_shape(self):
        return self.images.shape

    shape = property(_get_shape)

In [0]:
# Training parameters

def main(model_name='dcgan', epochs=100, batchsize=64, output_dir='MyDrive/DLtutorial/DCGAN/output', zdims=50, gpu=0):
    
    # select gpu
    os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu)

    # Make output direcotiry if not exists
    if not os.path.isdir(output_dir):
        os.mkdir(output_dir)

    datasets = load_data()

    model = models[model_name](
        input_shape=datasets.shape[1:],
        z_dims=zdims,
        output=output_dir
    )

    # Training loop
    datasets = datasets.images * 2.0 - 1.0
    samples = np.random.normal(size=(100, zdims)).astype(np.float32)
    model.main_loop(datasets, samples,
        epochs=epochs,
        batchsize=batchsize,
        reporter=['loss', 'g_loss', 'd_loss', 'g_acc', 'd_acc'])

**Some notes**

You can change the `zdim` parameter which decide how big is the sampling vector for generator input. The higher 'zdim', the more detailed image is. Higher 'zdim' also come with longer converging time. For example, while the `zdim = 100` will generate noise images for the first 160,000 samples (of total 202599), and start converging at the end of epoch 1,  'zdim = 50' shows face-like images at the first 40,000 samples.

**Troubleshooting**

- When running the code, it will output as:

`Epoch #1 | 17920/202599 (8.85 %) | g_loss = 1.943672 | d_loss = 0.776618 | g_acc = 0.031250 | d_acc = 0.539062 | ETA: 14 min 12 sec `

where, `g_loss, d_loss, g_acc, d_acc` are generator and discriminator loss and accuracy, perspectively. While you should pay attention on the `g_loss, d_loss, d_acc`, the `g_acc` doesn't have much meaning.

There are chances that you will encounter some troubles where the **`g_loss, d_loss, d_acc` don't change over iterations**. The main reason for these failure cases is that your hyperparametes were initially bad randomized.

**Solution**

- This code has been verified as working. Ideally, the `g_loss, d_loss, g_acc, d_acc` would change over iterations and `d_acc`would be somewhere around `0.3-0.7` as below.

Epoch #2 | 156992 / 202599 ( 77.49 %) | g_loss = 0.711317 | d_loss = 0.694376 | g_acc = 0.140625 | d_acc = 0.476562 | ETA: 3 min 11 sec 

- The networks often converge after 1-2 first epochs, depending on how you set `zdim`. So if the numbers don't change or `d_acc = 1` after 5 epochs, **just restart the code and run it again**.
- Run the code a few times and you will get the hang of it.

########## HAVE FUN ##########

In [0]:
if __name__ == '__main__':
    main()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 50)           0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
model_1 (Model)                 (None, 32, 32, 3)    2894723     input_4[0][0]                    
__________________________________________________________________________________________________
model_2 (Model)                 (None, 1)            67131393    input_3[0][0]                    
                                                                 model_1[1][0]                    
__________

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


Epoch #1 | 202599 / 202599 (100.00 %) | g_loss = 0.738534 | d_loss = 0.696304 | g_acc = 0.205128 | d_acc = 0.435897 | ETA: 0 sec 
Epoch #2 | 202599 / 202599 (100.00 %) | g_loss = 0.689617 | d_loss = 0.713234 | g_acc = 0.589744 | d_acc = 0.371795 | ETA: 0 sec 
Epoch #3 | 202599 / 202599 (100.00 %) | g_loss = 0.719746 | d_loss = 0.700083 | g_acc = 0.128205 | d_acc = 0.346154 | ETA: 0 sec 
Epoch #4 | 193408 / 202599 ( 95.46 %) | g_loss = 0.962931 | d_loss = 0.681890 | g_acc = 0.015625 | d_acc = 0.531250 | ETA: 39 sec 